In [ ]:
import pandas as pd
from prophet import Prophet
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

# Función para cargar los datos y filtrar la serie de tiempo seleccionada
def cargar_y_preparar_datos(df, industry_type, column_name):
    try:
        df_filtered = df[df['industry'] == industry_type].copy()
        df_filtered = df_filtered[df_filtered[column_name] >= 0]  # Filtrar valores negativos
        df_prophet = df_filtered[['date', column_name]].rename(columns={'date': 'ds', column_name: 'y'})
        df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
        return df_prophet
    except Exception as e:
        print(f"Error al cargar y preparar los datos: {e}")
        return None

# Función para guardar el forecast en un archivo CSV
def guardar_predicciones(forecast, nombre_archivo="predicciones.csv"):
    try:
        forecast.to_csv(nombre_archivo, index=False)
        print(f"Predicciones guardadas en {nombre_archivo}")
    except Exception as e:
        print(f"Error al guardar las predicciones: {e}")

# Cargar los datos
try:
    df = pd.read_csv("datasets/Depurados/TLC Aggregated Data/merged_taxi_data.csv")
except Exception as e:
    print(f"Error al cargar el archivo CSV: {e}")

# Definir parámetros de selección para la industria y la columna
industry_type = input(f"Seleccione el tipo de industria para predecir {tuple(df['industry'].unique())}: ")
column_name = input(f"Seleccione la variable que desea predecir ('trips_per_day', 'total_co2_emission', 'vehicles_per_day', 'total_trips', 'total_amount', 'farebox_per_day_per_distance', 'farebox_per_day', 'unique_drivers', 'unique_vehicles', 'avg_trip_distance'): ")

# Selección de períodos y frecuencia para predicción
try:
    periodos = int(input("Seleccione la cantidad de períodos para la predicción: "))
    frecuencia = input("Seleccione la frecuencia de predicción (D = diaria, W = semanal, M = mensual, YE = anual): ").upper()
    if frecuencia not in ["D", "W", "M", "YE"]:
        print("Frecuencia inválida, se usará la frecuencia diaria (D) por defecto.")
        frecuencia = "D"
except ValueError:
    print("Por favor, ingrese un número válido para los períodos.")
    periodos = 365  # Valor predeterminado
    frecuencia = "D"

# Preparar los datos para Prophet
df_prophet = cargar_y_preparar_datos(df, industry_type, column_name)

if df_prophet is not None:
    def graficar_original(df_prophet, column_name):
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], marker=dict(symbol='circle', color='royalblue')))
        fig.layout.update(title_text="Datos históricos", yaxis_title=f"{column_name}", xaxis_rangeslider_visible=True)
        fig.show()

    graficar_original(df_prophet, column_name)

    def predecir_columna(df_prophet, periodos, column_name, frecuencia):
        try:
            model = Prophet(changepoint_prior_scale=0.04, seasonality_prior_scale=10.0, seasonality_mode='additive')
            model.add_seasonality(name='monthly', period=12, fourier_order=2)
            model.fit(df_prophet)

            # Crear el futuro dataframe con frecuencia específica
            future = model.make_future_dataframe(periods=periodos, freq=frecuencia)
            forecast = model.predict(future)

            # Ajustar valores negativos en el forecast
            forecast['yhat'] = forecast['yhat'].clip(lower=0)
            forecast['yhat_lower'] = forecast['yhat_lower'].clip(lower=0)

            fig = go.Figure()
            fig.add_trace(go.Scatter(x=df_prophet["ds"], y=df_prophet["y"], name='Datos Históricos', marker=dict(symbol='circle', color='royalblue')))
            fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicción', marker=dict(symbol='diamond', color='yellow')))
            fig.add_trace(go.Scatter(x=forecast['ds'].tolist() + forecast['ds'][::-1].tolist(), 
                                     y=forecast['yhat_upper'].tolist() + forecast['yhat_lower'][::-1].tolist(), 
                                     fill='toself', fillcolor='rgba(255, 255, 255, 0.2)', 
                                     line=dict(color='rgba(255, 255, 255, 0)'), name='Intervalo de predicción'))
            fig.layout.update(xaxis_title='Fecha', yaxis_title=column_name, title_text=f"Predicción para {column_name} ({industry_type})", 
                              xaxis_rangeslider_visible=True)
            fig.show()

            return model, forecast
        except Exception as e:
            print(f"Error al realizar la predicción: {e}")
            return None, None

    model, forecast = predecir_columna(df_prophet, periodos, column_name, frecuencia)

    if model is not None and forecast is not None:
        fig1 = model.plot_components(forecast)
        plt.show()

        # Guardar las predicciones en un archivo CSV
        guardar_predicciones(forecast, f"predicciones_{industry_type}_{column_name}.csv")
else:
    print("No se pudieron preparar los datos para la predicción.")
